In [6]:
import cv2
from google.colab.patches import cv2_imshow
from darknet import *

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights


In [ ]:
%cd darknet

In [3]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

In [7]:
network, class_names, class_colors = load_network("cfg/yolov4.cfg", "cfg/coco.data", "yolov4.weights")
width = network_width(network)
height = network_height(network)

In [22]:
def darknet_helper(img, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_LINEAR)

    # Obtém a razão da imagem para ajustar as caixas delimitadoras
    img_height, img_width, _ = img.shape
    width_ratio = img_width / width
    height_ratio = img_height / height

    # Executa a detecção
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image)
    free_image(darknet_image)

    return detections, width_ratio, height_ratio

In [17]:
def detect_objects(image_path):
    # Carrega a imagem
    image = cv2.imread(image_path)

    # Verifiqua se a imagem foi carregada corretamente
    if image is None:
        print("Erro ao carregar a imagem.")
        return

    # Executa a detecção de objetos
    detections, width_ratio, height_ratio = darknet_helper(image, width, height)

    for label, confidence, bbox in detections:
        # Converte a confiança para float antes de formatá-la
        confidence = float(confidence)

        left, top, right, bottom = bbox2points(bbox)
        left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)

        # Desenha o retângulo e adicione o texto da detecção
        cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
        cv2.putText(image, f"{label} [{confidence:.2f}]", (left, top - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_colors[label], 2)

    cv2_imshow(image)


In [ ]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/horses.jpg -O horses.jpg
detect_objects("horses.jpg")